# Data-X Spring 2019: Homework 7

### Webscraping



In this homework, you will do some exercises with web-scraping.

## Name: Wesley Chen

## SID: 3031961388



### Fun with Webscraping & Text manipulation


## 1. Statistics in Presidential Debates

Your first task is to scrape Presidential Debates from the Commission of Presidential Debates website: https://www.debates.org/voter-education/debate-transcripts/

To do this, you are not allowed to manually look up the URLs that you need, instead you have to scrape them. The root url to be scraped is the one listed above, namely: https://www.debates.org/voter-education/debate-transcripts/


1. By using `requests` and `BeautifulSoup` find all the links / URLs on the website that links to transcriptions of **First Presidential Debates** from the years [1988, 1984, 1976, 1960]. In total you should find 4 links / URLs that fulfill this criteria. **Print the urls.**

2. When you have a list of the URLs your task is to create a Data Frame with some statistics (see example of output below):
    1. Scrape the title of each link and use that as the column name in your Data Frame. 
    2. Count how long the transcript of the debate is (as in the number of characters in transcription string). Feel free to include `\` characters in your count, but remove any breakline characters, i.e. `\n`. You will get credit if your count is +/- 10% from our result.
    3. Count how many times the word **war** was used in the different debates. Note that you have to convert the text in a smart way (to not count the word **warranty** for example, but counting **war.**, **war!**, **war,** or **War** etc.
    4. Also scrape the most common used word in the debate, and write how many times it was used. Note that you have to use the same strategy as in C in order to do this.
    
    **Print your final output result.**
    
**Tips:**

___

In order to solve the questions above, it can be useful to work with Regular Expressions and explore methods on strings like `.strip(), .replace(), .find(), .count(), .lower()` etc. Both are very powerful tools to do string processing in Python. To count common words for example I used a `Counter` object and a Regular expression pattern for only words, see example:

```python
    from collections import Counter
    import re

    counts = Counter(re.findall(r"[\w']+", text.lower()))
```

Read more about Regular Expressions here: https://docs.python.org/3/howto/regex.html
    
    
**Example output of all of the answers to Question 1.2:**


![pres_stats_2](https://github.com/ikhlaqsidhu/data-x/raw/master/x-archive/misc/hw2_imgs_spring2018/presidents_stats_2.jpg)



----

.




In [75]:
from __future__ import division, print_function
from collections import Counter

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import requests 
import bs4 as bs
import numpy as np
import pandas as pd
import re

In [4]:
source = requests.get("https://www.debates.org/voter-education/debate-transcripts/") 
print(source)

<Response [200]>


In [24]:
#1
soup = bs.BeautifulSoup(source.content, features='html.parser')
links = []
for url in soup.find_all('a'): 
    link = url.get('href')
    if '1988' in link:
        links.append(link)
    if '1984' in link:
        links.append(link)
    if '1976' in link:
        links.append(link)
    if '1960' in link:
        links.append(link)
first_pres_debates = np.unique(links)[9:]
valid_url = ['https://www.debates.org' + url for url in first_pres_debates]
valid_url

['https://www.debates.org/voter-education/debate-transcripts/october-7-1984-debate-transcript/',
 'https://www.debates.org/voter-education/debate-transcripts/september-23-1976-debate-transcript/',
 'https://www.debates.org/voter-education/debate-transcripts/september-25-1988-debate-transcript/',
 'https://www.debates.org/voter-education/debate-transcripts/september-26-1960-debate-transcript/']

In [45]:
#2a
debate_1984 = bs.BeautifulSoup(requests.get(valid_url[0]).content, features='html.parser')
debate_1976 = bs.BeautifulSoup(requests.get(valid_url[1]).content, features='html.parser')
debate_1988 = bs.BeautifulSoup(requests.get(valid_url[2]).content, features='html.parser')
debate_1960 = bs.BeautifulSoup(requests.get(valid_url[3]).content, features='html.parser')

def get_title(soup):
    title = []
    for x in soup.find_all('strong'):
        title.append(x.text)
    return title[0]+': '+ title[1]
titles = [get_title(debate_1988), get_title(debate_1984), get_title(debate_1976), get_title(debate_1960)]
titles

['September 25, 1988: The First Bush-Dukakis Presidential Debate',
 'October 7, 1984: The First Reagan-Mondale Presidential Debate',
 'September 23, 1976: The First Carter-Ford Presidential Debate',
 'September 26, 1960: The First Kennedy-Nixon Presidential Debate']

In [59]:
#2b
def get_char_count(soup):
    body = []
    for x in soup.find_all('p'):
        body.append(x.text)
    transcription = body[2:]
    debate_char_count = 0
    for x in transcription:
        debate_char_count += len(x) 
    return debate_char_count
char_count = [get_char_count(debate_1988),get_char_count(debate_1984), get_char_count(debate_1976), get_char_count(debate_1960)]
char_count

[87469, 86490, 80717, 60917]

In [128]:
#2c
def get_transcription(soup):
    body = []
    for x in soup.find_all('p'):
        body.append(x.text)
    transcription = body[2:]
    trans_combined = ''
    for x in transcription:
        trans_combined += x + ' '
    return trans_combined

t_1988 = get_transcription(debate_1988)
t_1984 = get_transcription(debate_1984)
t_1976 = get_transcription(debate_1976)
t_1960 = get_transcription(debate_1960)


def get_war_count(text):
    lower = text.lower()
    no_punc = re.sub(r'[^\w\s]',' ', lower)
    return len(re.findall(r'\bwar\b', no_punc))

war_count = [get_war_count(t_1988), get_war_count(t_1984), get_war_count(t_1976), get_war_count(t_1960)]

war_count

[8, 2, 7, 3]

In [158]:
#2d

c_1988 = Counter(re.findall(r"[\w']+", t_1988.lower())).most_common(1)[0]
c_1984 = Counter(re.findall(r"[\w']+", t_1984.lower())).most_common(1)[0]
c_1976 = Counter(re.findall(r"[\w']+", t_1976.lower())).most_common(1)[0]
c_1960 = Counter(re.findall(r"[\w']+", t_1960.lower())).most_common(1)[0]

most_c_word = [c_1988[0], c_1984[0], c_1976[0], c_1960[0]]
most_c_word_count = [c_1988[1], c_1984[1], c_1976[1], c_1960[1]]
print(most_c_word_count)
print(most_c_word)

[804, 867, 857, 779]
['the', 'the', 'the', 'the']


In [236]:
pres_debates= pd.DataFrame(np.array([char_count, war_count,most_c_word, most_c_word_count]),
                           columns = titles,
                           index = ['Debate char length', 'War count', 'Most common word', 'Most common word count'])
display(pres_debates)

,"September 25, 1988: The First Bush-Dukakis Presidential Debate","October 7, 1984: The First Reagan-Mondale Presidential Debate","September 23, 1976: The First Carter-Ford Presidential Debate","September 26, 1960: The First Kennedy-Nixon Presidential Debate"
Debate char length,87469,86490,80717,60917
War count,8,2,7,3
Most common word,the,the,the,the
Most common word count,804,867,857,779


    
## 2. Download and read in specific line from many data sets

Scrape the first 27 data sets from this URL http://people.sc.fsu.edu/~jburkardt/datasets/regression/ (i.e.`x01.txt` - `x27.txt`). Then, save the 5th line in each data set, this should be the name of the data set author (get rid of the `#` symbol, the white spaces and the comma at the end). 

Count how many times (with a Python function) each author is the reference for one of the 27 data sets. Showcase your results, sorted, with the most common author name first and how many times he appeared in data sets. Use a Pandas DataFrame to show your results, see example. **Print your final output result.**

**Example output of the answer for Question 2:**

![author_stats](https://github.com/ikhlaqsidhu/data-x/raw/master/x-archive/misc/hw2_imgs_spring2018/data_authors.png)


In [200]:
def get_author(txt):
    raw = requests.get('http://people.sc.fsu.edu/~jburkardt/datasets/regression/'+ txt).content
    soup = bs.BeautifulSoup(raw,features='html.parser')
    line = str(soup.prettify().split('\n')[4])
    return re.sub(r' {2,}', '', line).strip('#').strip(',')

txt_files = ['x01.txt', 'x02.txt', 'x03.txt', 'x04.txt', 'x05.txt', 'x06.txt', 'x07.txt', 'x08.txt', 'x09.txt', 'x10.txt',
             'x11.txt','x12.txt', 'x13.txt', 'x14.txt', 'x15.txt', 'x16.txt', 'x17.txt', 'x18.txt', 'x19.txt', 'x20.txt',
             'x21.txt', 'x22.txt', 'x23.txt', 'x24.txt', 'x25.txt', 'x26.txt', 'x27.txt']
authors = list()
for x in txt_files:
    authors.append(get_author(x))
authors

['Helmut Spaeth',
 'Helmut Spaeth',
 'Helmut Spaeth',
 'Helmut Spaeth',
 'Helmut Spaeth',
 'R J Freund and P D Minton',
 'D G Kleinbaum and L L Kupper',
 'Helmut Spaeth',
 'D G Kleinbaum and L L Kupper',
 'K A Brownlee',
 'Helmut Spaeth',
 'Helmut Spaeth',
 'S Chatterjee and B Price',
 'Helmut Spaeth',
 'Helmut Spaeth',
 'Helmut Spaeth',
 'Helmut Spaeth',
 'Helmut Spaeth',
 'R J Freund and P D Minton',
 'Helmut Spaeth',
 'Helmut Spaeth',
 'Helmut Spaeth',
 'S Chatterjee, B Price',
 'S Chatterjee, B Price',
 'S Chatterjee, B Price',
 'S C Narula, J F Wellington',
 'S C Narula, J F Wellington']

In [235]:
count_authors = Counter(authors)

author_df = pd.DataFrame(np.array(count_authors.most_common()), columns = ['Authors', 'Count'])
author_df.set_index('Authors', inplace = True)

display(author_df)

,Count
Authors,
Helmut Spaeth,16
"S Chatterjee, B Price",3
R J Freund and P D Minton,2
D G Kleinbaum and L L Kupper,2
"S C Narula, J F Wellington",2
K A Brownlee,1
S Chatterjee and B Price,1
